# Генетический алгоритм для минимизации функции
# genetic algorithm search for continuous function optimization

In [2]:
from numpy.random import randint
from numpy.random import rand

In [6]:
# objective function
def objective(x):
	return (1.5 - x[0])**2.0 + (-2.5 - x[1])**2.0 + (-2.5 - x[2])**2.0
 
# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
	decoded = list()
	largest = 2**n_bits
	for i in range(len(bounds)):
		# extract the substring
		start, end = i * n_bits, (i * n_bits)+n_bits
		substring = bitstring[start:end]
		# convert bitstring to a string of chars
		chars = ''.join([str(s) for s in substring])
		# convert string to integer
		integer = int(chars, 2)
		# scale integer to desired range
		value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
		# store
		decoded.append(value)
	return decoded
 
# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]
 
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]
 
# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]
 
# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
	# enumerate generations
	for gen in range(n_iter):
		# decode population
		decoded = [decode(bounds, n_bits, p) for p in pop]
		# evaluate all candidates in the population
		scores = [objective(d) for d in decoded]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval:
				best, best_eval = pop[i], scores[i]
				print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]
 
# define range for input
bounds = [[-10.0, 10.0], [-10.0, 10.0]]
# define the total iterations
n_iter = 100
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))

>0, new best f([-1.416015625, 1.407470703125]) = 23.771474
>0, new best f([-3.14178466796875, -2.98614501953125]) = 21.782502
>0, new best f([-0.999755859375, -3.9794921875]) = 8.437676
>0, new best f([1.73126220703125, -2.16949462890625]) = 0.162716
>2, new best f([1.73126220703125, -2.17010498046875]) = 0.162313
>3, new best f([1.41204833984375, -2.11944580078125]) = 0.152557
>4, new best f([1.57501220703125, -2.12921142578125]) = 0.143111
>5, new best f([1.58111572265625, -2.63336181640625]) = 0.024365
>5, new best f([1.44500732421875, -2.554931640625]) = 0.006042
>7, new best f([1.48956298828125, -2.44598388671875]) = 0.003027
>12, new best f([1.50238037109375, -2.5177001953125]) = 0.000319
>14, new best f([1.5093994140625, -2.51068115234375]) = 0.000202
>17, new best f([1.50604248046875, -2.51007080078125]) = 0.000138
>18, new best f([1.4984130859375, -2.50823974609375]) = 0.000070
>23, new best f([1.49505615234375, -2.50518798828125]) = 0.000051
>24, new best f([1.49505615234375,